# 多量子比特含噪模拟器
*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

在本教程中，我们将介绍如何使用脉冲层面的多量子比特模拟器。本教程的概要如下：

- 背景介绍
- 准备工作
- 含噪模拟器的门级控制
- 含噪模拟器的脉冲级控制
    - 构建系统的物理模型
    - 拉比振荡
    - Cross-Resonance 效应
    - 通过 Ramsey 实验刻画 ZZ 串扰
- 总结

## 背景介绍

在脉冲层面模拟量子比特的演化能让我们更深入地了解量子门的工作原理与噪声的来源。在超导电路中，transmon 量子比特通过施加微波或磁通脉冲来控制。然而，量子门的性能通常受到各种因素的影响——量子比特与环境相互作用导致的退相干、串扰噪声以及向高能级的泄漏。

量脉提供的多比特含噪模拟器使我们能够在多量子比特的含噪声设备上模拟量子操作，以更好地了解量子计算背后的物理原理。该含噪模拟器包含了几种主要的噪声：退相干噪声、脉冲幅值失真噪声以及串扰噪声。基于该含噪模拟器我们将重点呈现超导量子计算测控流程中几项常见的操作，包括：拉比振荡实验，Cross-Resonance 效应以及通过 Ramsey 实验刻画 ZZ 串扰。

## 准备工作

成功安装量脉后，您可以按照本教程运行下面的量脉程序。要运行此教程，您需要从 Quanlse 和其它常用的 Python 库导入以下包：

In [ ]:
from Quanlse.remoteOptimizer import remoteOptimize1Qubit as optimize1q
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian
from Quanlse.Superconduct.Simulator import PulseModel
from Quanlse.Superconduct.Simulator.PulseSim3Q import pulseSim3Q

from Quanlse.QWaveform import QJob, QJobList
from Quanlse.QOperator import driveX, driveY, a, sigmaZ, number, driveZ
from Quanlse.QWaveform import square, gaussian
from Quanlse.Utils.Functions import basis, tensor, expect, dagger, partialTrace, project, computationalBasisList, population
from Quanlse.Utils.Bloch import rho2Coordinate, plotBloch
from Quanlse.Utils.Plot import plotBarGraph
from Quanlse.QOperation.FixedGate import H, X, Y, CR, Z, CNOT
from Quanlse.QOperation.RotationGate import RZ
from Quanlse.Superconduct.SchedulerSupport import SchedulerSuperconduct
from Quanlse.Utils.Infidelity import unitaryInfidelity
from Quanlse.Superconduct.SchedulerSupport.PipelineCenterAligned import centerAligned

from math import pi
from scipy.optimize import curve_fit

import numpy as np
import matplotlib.pyplot as plt

使用量脉云服务之前，我们需要一个 token 来访问云端。

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## 含噪模拟器的门级控制


我们可以在量子门层面进行模拟。在这一部分，我们需要使用预先定义、具有默认配置的 `PulseModel()` 对象。

为了创建一个三量子比特物理系统，我们首先通过调用 `pulseSim3Q()` 来实例化 `PulseModel()` 对象。

In [ ]:
model = pulseSim3Q(frameMode='lab', dt=0.01)
model.savePulse = False
model.pipeline.addPipelineJob(centerAligned)

为了使用量脉调度器来定义一个产生 GHZ 态的量子电路， 我们可以通过调用 `gate(model.Q[index])` 将这些量子门添加到物理模型中。

In [ ]:
# Hadamard gate 
H(model.Q[0])

# CNOT
CNOT(model.Q[0], model.Q[1])
CNOT(model.Q[1], model.Q[2])

上面定义的量子电路的脉冲序列是通过调用函数 `model.schedule` 生成的。

In [ ]:
scheJob = model.schedule()

定义该三个量子比特初始态都在基态，也就是系统初始态为 $|\psi\rangle = |000\rangle$，然后运行模拟，并且画出每个测量结果的概率分布。

In [ ]:
# Run the simulation
res = model.simulate(job=scheJob)

In [ ]:
# Plot the result
popList = [abs(item ** 2) for item in res[0]['state'].T[0]]
basisList = computationalBasisList(3, 3)
plotBarGraph(basisList, popList, "Result", "Outcome", "Population")

由上图可见，由于噪声的存在，测量结果会得到 $000$ 和 $111$ 外的值。使用含噪模拟器研究退相干情况下的模拟，用户可以尝试设置 `runHamiltonian()` 里的参数 `isOpen` 为 `True`，得到演化过程的密度矩阵，但是会花费一定的时间得到结果。如果想知道退相干如何影响超导量子计算，可以参考教程：[单量子比特含噪模拟器](https://quanlse.baidu.com/#/doc/tutorial-single-qubit-noisy-simulator)。

## 含噪模拟器的脉冲级控制

含噪模拟器支持脉冲级的模拟。通过对量子比特 $q_i$ 输入脉冲波形以及其它参数，就能得到系统的动力学演化，这使得我们可以从更底层模拟超导量子计算的控制环节。这里我们利用该模拟器，演示几个真实实验中常用的操作。

### 构建系统的物理模型

通常，我们可以用 Duffing 振子模型来描述超导量子比特的物理模型。以三比特模型为例，当量子比特 $q_0$ 与 $q_1$、$q_1$ 与 $q_2$ 之间有耦合的时候，在实验坐标系，哈密顿量可以表示为:

$$
\hat{H} = \sum_{i=0}^2 \omega_i \hat{a}^\dagger_i \hat{a}_i + \sum_{i=0}^2 \frac{\alpha_i}{2} \hat{a}^\dagger_i \hat{a}^\dagger_i \hat{a}_i \hat{a}_i + g_{01} (\hat{a}^\dagger_0 \hat{a}_1 + \hat{a}_0 \hat{a}^\dagger_1) +  g_{12} (\hat{a}^\dagger_1 \hat{a}_2 + \hat{a}_1 \hat{a}^\dagger_2),
$$

其中，参数 $\omega_i$、$\alpha_i$ 分别是量子比特 $q_i$ 的本征频率以及失谐量；而 $g_{ij}$ 则是量子比特 $q_i$ 以及量子比特 $q_j$ 之间的耦合强度。算符 $\hat{a}_i$、$\hat{a}^\dagger_i$ 是量子比特 $q_i$ 的湮灭算符和产生算符。

在本教程中，我们构造了一个包含三个量子比特的含噪模拟器。首先，我们定义一些硬件参数：

In [ ]:
qubitNum = 3  # The number of qubits
level = 3  # The energy level for each qubit

anharm = -0.33 * 2 * pi  # The anharmonicity of the qubit, in 2 pi GHz
wq0 = 4.914 * 2 * pi  # The frequency for qubit 0, in 2 pi GHz 
wq1 = 5.100 * 2 * pi  # The frequency for qubit 1, in 2 pi GHz
wq2 = 5.200 * 2 * pi  # The frequency for qubit 2, in 2 pi GHz
g01 = 0.0038 * 2 * pi  # The coupling strength of the interaction between qubit 0 and qubit 1, in 2 pi GHz
g12 = 0.0020 * 2 * pi  # The coupling strength of the interaction between qubit 1 and qubit 2, in 2 pi GHz

dt = 1.  # The sampling time of AWG

# T1 relaxation time for qubit 0, qubit 1, and qubit 2, in nanoseconds
t01 = 1000  
t11 = 1120
t21 = 1300

# T2 dephasing time for qubit 0, qubit 1, and qubit 2, in nanoseconds
t02 = 500
t12 = 450
t22 = 600

# The random amplitude distortion
ampNoise = 0.02

如下，我们可以通过实例化 `PulseModel` 类中的一个对象来创建该物理系统。物理系统中的噪声包含 $T_1$ 弛豫噪声, $T_2$ 失相噪声与振幅噪声。

In [ ]:
qubitFreq = {0: wq0, 1: wq1, 2: wq2}  # Qubit frequency for each qubit
qubitAnharm = {0: anharm, 1: anharm, 2: anharm}  # Qubit anharmonicity for each qubit
qubitT1 = {0: t01, 1: t11, 2: t21}  # Relaxation time 
qubitT2 = {0: t02, 1: t12, 2: t22}  # Dephasing time
couplingMap = {(0, 1): g01, (1, 2): g12}  # Coupling map

# Create an instant of PulseModel
model = PulseModel(subSysNum=qubitNum,
                   sysLevel=level,
                   qubitFreq=qubitFreq,
                   qubitAnharm=qubitAnharm,
                   couplingMap=couplingMap,
                   T1=qubitT1,
                   T2=qubitT2,
                   dt=dt,
                   ampSigma=ampNoise)

于是一个包含三个超导量子比特和三种常见噪声的模拟器就构建完成了。下一步，我们通过 `createQHamiltonian()` 方法创建一个 `QHamiltonian` 的对象。

In [ ]:
ham = model.createQHamiltonian()

### Cross-Resonance 效应

全微波控制是实现超导电路量子控制的方案之一。 在该方案中，双量子比特门的实现利用了两个弱耦合量子比特的 Cross-Resonance 效应。该效应通过在控制量子比特上施加以目标量子比特频率为驱动频率的脉冲完成。理想情况下，控制量子比特和目标量子比特之间的 $\hat{\sigma}_z \otimes \hat{\sigma}_x$ 相互作用在所有类型的相互作用中占主导地位 \[1\]。如果想知道更多关于 CR 门的内容，可以参考教程：[Cross-Resonance 门](https://quanlse.baidu.com/#/doc/tutorial-cr)。

在我们的模拟中，我们再次以不同的脉冲幅值驱动量子比特 $q_0$ （控制量子比特），脉冲的驱动频率为量子比特 1 的频率。这一过程可以由方法 `addWaveRot(index, waves, detuning)` 实现，其中 `index` 是作用的量子比特编号、`waves` 是脉冲波形、 `detuning` $\Delta$ 是频率差（$\Delta = \omega_i - \omega_d$，其中 $\omega_i$ 是量子比特 $q_i$ 的频率而 $\omega_d$ 是驱动频率）。 

接下来我们改变脉冲幅值，观察在 cross-resonance 效应下不同幅值下每个量子比特的布居数变化。本例以量子比特 $q_1$ 为控制比特，$q_0$ 为目标比特，也就是说用驱动频率是 $q_0$ 的本征频率的脉冲驱动控制比特 $q_1$。 

In [ ]:
dCoef = 0.03 * (2 * pi)  # The drive strength of the pulse
ampCR = np.linspace(0, 0.5, 40)  # The amplitudes in arbitrary unit 
amps = ampCR * dCoef  
detuning = wq1 - wq0  # The detuning of the pulse

# jobList = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt, title='cr')
jobList = ham.createJobList()

# Fix the gate time
tg = 950

# Append each job to the jobList
for amp in amps:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job = ham.createJob()
    job.addWaveRot(1, waves=square(tg, amp), t0=0., detuning=detuning)  # Apply pulse at qubit 1
    job = model.getSimJob(job)
    jobList.addJob(jobs=job)

现在，我们设置该三比特系统的初始态为 $|\psi\rangle = |010\rangle$，即控制比特 $q_1$ 处在激发态，然后进行模拟。

In [ ]:
# Define the initial state of |010>
psi0 = tensor(basis(level, 0), basis(level, 1), basis(level, 0))  

# Run the simulation
result = runHamiltonian(ham=ham, state0=psi0, jobList=jobList)  

定义每个量子比特 $q_i$ 上的第一激发态的投影算符。

In [ ]:
prj01 = tensor(basis(3, 1) @ dagger(basis(3,1)), np.identity(level), np.identity(level))  
prj11 = tensor(np.identity(level), basis(3, 1) @ dagger(basis(3,1)), np.identity(level))  
prj21 = tensor(np.identity(level), np.identity(level), basis(3, 1) @ dagger(basis(3,1)))  

计算每个量子比特上投影算符 $|1\rangle\langle1|$ 的期望值，然后绘制激发态布居数随脉冲幅值变化的图。

In [ ]:
# Initialize the list of expected values
num0List = []
num1List = []
num2List = []

for res in result.result:
    state = res['state']  # The final state of each job
    num0Expect = expect(prj01, state)  # Compute the expected values of the projector |1><1|
    num1Expect = expect(prj11, state)
    num2Expect = expect(prj21, state)
    num0List.append(num0Expect)
    num1List.append(num1Expect)
    num2List.append(num2Expect)

plt.figure(figsize=(8, 6))
# Plot the figure of CR effect
plt.plot(ampCR, num0List, label='qubit0')
plt.plot(ampCR, num1List, label='qubit1')
plt.plot(ampCR, num2List, label='qubit2')

plt.xlabel('Amplitudes (a.u.)')
plt.ylabel(r'Population of $|1\rangle$')
plt.title('Cross-Resonance effect')
plt.legend()
plt.show()

可见，目标比特 $q_0$ 的激发态布居数随着脉冲幅度增加而改变，而控制比特 $q_1$ 在脉冲幅度较小的时候基本处于激发态，与 $q_1$ 耦合的比特 $q_2$ 则一直处于基态。当脉冲幅度进一步增大的时候，也会不可避免地影响到到量子比特 $q_1$。

### 通过 Ramsey 实验刻画 ZZ 串扰

ZZ 串扰是耦合量子比特间寄生相互作用的主要来源，它源于量子态更高能级间的相互作用。对于一对直接或者间接耦合的量子比特，其计算空间的有效哈密顿量为 \[2\]：

$$
\hat{H}_{\rm eff} = \omega_{0}\frac{\hat{\sigma}_{z}^0 \otimes I_1}{2} + \omega_{1}\frac{I_0\otimes\hat{\sigma}_{z}^1}{2} + \xi \frac{\hat{\sigma}_{z}^0 \otimes \hat{\sigma}_{z}^1}{2},
$$

其中 $\omega_0$，$\omega_1$ 是量子比特的频率，$\xi$ 是 ZZ 串扰的强度。$\xi$ 由量子态 $|11\rangle \leftrightarrow |10\rangle$ 与 $|01\rangle \leftrightarrow |00\rangle$ 间的跃迁频率定义:

$$
\xi = \left(E_{11} - E_{10}\right) - \left(E_{01} - E_{00}\right),
$$

其中，$E_{ij}$ 是态 $|ij\rangle$ 的能级。实际上，我们可以通过 Ramsey 实验探测到这个频率差，因此对该串扰进行度量与刻画。在逻辑量子门层面，这相当于在两个 $H$ 门间有一段空转的时间 \[3\]。

为了更好地描述 ZZ 串扰带来的影响，我们定义一个耦合强度更大的三量子比特模型（耦合强度约为 6 ~ 40 MHz）。

In [ ]:
dt = 0.2  # The sampling time
level = 3  # The system level
qubitNum = 3  # The number of qubits

g01 = 0.0377 * (2 * pi)
g12 = 0.0060 * (2 * pi)

# Coupling map
couplingMap = {
    (0, 1): g01,
    (1, 2): g12
}

# Qubits frequency anharmonicity
anharm = - 0.33 * (2 * pi)
qubitAnharm = {0: anharm, 1: anharm, 2: anharm}  # The anharmonicities for each qubit

# Qubit Frequency
qubitFreq = {
            0: 5.5904 * (2 * pi),
            1: 4.7354 * (2 * pi),
            2: 4.8524 * (2 * pi)
            }

通过类 `PulseModel()` 创建物理模型，并且根据模型创建哈密顿量 `ham`。

In [ ]:
model = PulseModel(subSysNum=qubitNum, sysLevel=level, couplingMap=couplingMap,
                    qubitFreq=qubitFreq, dt=dt, qubitAnharm=qubitAnharm)
    
ham = model.createQHamiltonian()

通过调用单量子比特门的优化器, 生成作用在不同量子比特的 $H$ 门与 $X$ 门。

In [ ]:
# Define function to generate the QJob for gate of specified qubit 
def generateGate(gate, index):
    job1q, _ = optimize1q(ham=ham.subSystem(index), uGoal=gate.getMatrix(), targetInfid=1e-5)
    job3q = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    waves = job1q.waves
    ops = job1q.ctrlOperators
    for key, op in ops.items():
        job3q.addWave(operators=op, onSubSys=index, waves=waves[key])
     
    return job3q

# Generate the gates needed
h0 = generateGate(H, 0)  # H gate on qubit 0 
h1 = generateGate(H, 1)  # H gate on qubit 1
x1 = generateGate(X, 1)  # X gate on qubit 1
x2 = generateGate(X, 2)  # X gate on qubit 2

In [ ]:
maxTime = 500  # The delayed time in Ramsey experiment, in nanosecond.
freq = 3 / maxTime  # Detuning. 

# Generate job for delayed time 
def generateIdle(tg, index):
    jobIdle = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    jobIdle.appendWave(operators=driveZ, onSubSys=index, waves=square(tg, 2 * pi * freq))
    return jobIdle

定义两个不同的 `jobList` 对象——一个初始量子态是 $|00\rangle$，另一个通过作用一个 $X$ 门到量子比特 $q_1$ 上将其初始态是 $|01\rangle$。然后在量子比特 $q_0$ 上进行 Ramsey 实验。

In [ ]:
# jobList with initial state |00>
jobListGrd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# jobList with initial state |01> (by applying X gate) 
jobListExd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# Define the delayed time
tgList = np.linspace(0, maxTime, 50)

# Define jobList with initial state |00>
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += h0
    job += generateIdle(tg, 0)
    job += h0
    jobListGrd.addJob(job)

# Define jobList with initial state |01>
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += x1
    job += h0
    job += generateIdle(tg, 0)
    job += h0
    jobListExd.addJob(job)

# Run the simulation
stateInit = tensor(basis(level, 0), basis(level, 0), basis(level, 0))
resultGrd = runHamiltonian(ham, state0=stateInit, jobList=jobListGrd)
resultExd = runHamiltonian(ham, state0=stateInit, jobList=jobListExd)

绘制量子比特 $q_0$ 的第一激发态的布居数随空转时间变化的分布。

In [ ]:
num0List = []
num1List = []

# projector |1><1| of qubit 0
prj1 = tensor(basis(level, 1) @ dagger(basis(level, 1)), np.identity(9))

# append the result to the list
for res0, res1 in zip(resultGrd, resultExd):
    psi0, psi1 = res0['state'], res1['state']
    rho0, rho1 = psi0 @ dagger(psi0), psi1 @ dagger(psi1)
    num0List.append(expect(prj1, rho0))
    num1List.append(expect(prj1, rho1))


# plot the result
plt.figure(figsize=(8, 6))
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 0')
plt.title('Ramsey on Q0')
plt.legend()
plt.show()

ZZ 串扰的强度可以通过不同的 Ramsey 振荡频率测量。因此，我们通过用余弦函数去拟合得到的结果，并且计算得到频率 $f_1$，$f_2$。串扰强度为 $\xi / \left( 2\pi \right) = |f_1 - f_2|$。

In [ ]:
# Define the fitting curve
def fit(x, omega, theta):
    return - 0.5 * np.cos(omega * x + theta) + 0.5

# Fit the curve
para1Fit, _ = curve_fit(fit, tgList, num0List, [2.1 * pi * freq, 0])
para2Fit, _ = curve_fit(fit, tgList, num1List, [2 * pi * freq, 0])
step = 0.01
y1Fit = [fit(x, para1Fit[0], para1Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]
y2Fit = [fit(x, para2Fit[0], para2Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]

plt.figure(figsize=(8, 6))
# Plot the curve
plt.plot(np.arange(tgList[0], tgList[-1], step), y1Fit)
plt.plot(np.arange(tgList[0], tgList[-1], step), y2Fit)
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 0')
plt.title('Ramsey on Q0')
plt.legend()
plt.show()

In [ ]:
# Calculate the crosstalk strength
xiEst = abs(para1Fit[0] - para2Fit[0]) 
print(f'Coupling strength: {g01 * 1e3 / (2 * pi)} MHz')
print(f'ZZ crosstalk strength: {xiEst * 1e3 / (2 * pi)} MHz')

由于量子比特 $q_0$ 以及 $q_1$ 之间存在比较强的耦合强度，能观察到频率差比较大，也就是 ZZ 串扰比较大。

我们可以验证较小的耦合强度下的 ZZ 串扰也会较小。重复模拟该实验以得到量子比特 $q_1$ 与 $q_2$ 间的 ZZ 串扰强度。

In [ ]:
# jobList with initial state |00>
jobListGrd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# jobList with initial state |01> (by applying X gate)
jobListExd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# Define the delayed time
tgList = np.linspace(0, maxTime, 50)

# Define jobList with initial state |00>
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += h1
    job += generateIdle(tg, 1)
    job += h1
    jobListGrd.addJob(job)

# Define jobList with initial state |01>    
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += x2
    job += h1
    job += generateIdle(tg, 1)
    job += h1
    jobListExd.addJob(job)

# Run the simulation    
stateInit = tensor(basis(level, 0), basis(level, 0), basis(level, 0))
resultGrd = runHamiltonian(ham, state0=stateInit, jobList=jobListGrd)
resultExd = runHamiltonian(ham, state0=stateInit, jobList=jobListExd)

In [ ]:
num0List = []
num1List = []

# projector |1><1| of qubit 1
prj1 = tensor(np.identity(3), basis(level, 1) @ dagger(basis(level, 1)), np.identity(3))

# append the result to the list
for res0, res1 in zip(resultGrd, resultExd):
    psi0, psi1 = res0['state'], res1['state']
    rho0, rho1 = psi0 @ dagger(psi0), psi1 @ dagger(psi1)
    num0List.append(expect(prj1, rho0))
    num1List.append(expect(prj1, rho1))


# plot the result
plt.figure(figsize=(8, 6))
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 1')
plt.xlabel
plt.title('Ramsey on Q1')
plt.legend()
plt.show()

In [ ]:
# Fit the curve
para1Fit, _ = curve_fit(fit, tgList, num0List, [2 * pi * freq / 1.2, 0])
para2Fit, _ = curve_fit(fit, tgList, num1List, [2 * pi * freq / 1.2, 0])
step = 0.01
y1Fit = [fit(x, para1Fit[0], para1Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]
y2Fit = [fit(x, para2Fit[0], para2Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]

# Plot the curve
plt.figure(figsize=(8, 6))
plt.plot(np.arange(tgList[0], tgList[-1], step), y1Fit)
plt.plot(np.arange(tgList[0], tgList[-1], step), y2Fit)
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 1')
plt.xlabel
plt.title('Ramsey on Q1')
plt.legend()
plt.show()

In [ ]:
## Calculate the crosstalk strength
xiEst = abs(para1Fit[0] - para2Fit[0]) 
print(f'Coupling strength: {g12 * 1e3 / (2 * pi)} MHz')
print(f'ZZ crosstalk strength: {xiEst * 1e3 / (2 * pi)} MHz')

从上图可知，因为 $q_1$ 和 $q_2$ 之间耦合强度较小，所以当 $q_2$ 分别处于基态和激发态的时候对应的 $q_1$ 的量子比特频率的差距也不大，说明 $q_1$ 与 $q_2$ 之间的 ZZ 串扰比 $q_0$ 和 $q_1$ 之间的较小。

## 总结

阅读此关于多比特含噪模拟器的教程后，用户可以通过点击这个链接 [tutorial-multi-qubit-noisy-simulator.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-multi-qubit-noisy-simulator-cn.ipynb) 跳转到此 Jupyter Notebook 文档相应的 GitHub 页面，并且运行这个程序。我们鼓励用户使用多比特含噪模拟器探索与本教程不同的其他案例。

## 参考文献
\[1\] [Malekakhlagh, Moein, Easwar Magesan, and David C. McKay. "First-principles analysis of cross-resonance gate operation." *Physical Review A* 102.4 (2020): 042605.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.102.042605)

\[2\] [Magesan, Easwar, and Jay M. Gambetta. "Effective Hamiltonian models of the cross-resonance gate." *Physical Review A* 101.5 (2020): 052308.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.052308)

\[3\] [Ku, Jaseung, et al. "Suppression of Unwanted ZZ Interactions in a Hybrid Two-Qubit System." *Physical review letters* 125.20 (2020): 200504.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.200504)